##### Installing all the necessary libraries

In [23]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [24]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


In [25]:
pip install bs4

Note: you may need to restart the kernel to use updated packages.


In [26]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


##### Importing all the necessary libraries.

In [27]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


##### Web Scraping data from Wikipedia page.

In [40]:
# sending a GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_in_Kuala_Lumpur").text

In [41]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [42]:
# create a list to store neighborhood data
neighborhoodlist = []

In [43]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodlist.append(row.text)

In [44]:
# create a new DataFrame from the list
kl_df = pd.DataFrame({"Neighborhood": neighborhoodlist})
kl_df

,Neighborhood
0,Alam Damai
1,"Ampang, Kuala Lumpur"
2,Bandar Menjalara
3,Bandar Sri Permaisuri
4,Bandar Tasik Selatan
5,Bandar Tun Razak
6,Bangsar
7,Bangsar Park
8,Bangsar South
9,Batu 11 Cheras


##### Get the geographical coordinates.

In [45]:
# define a function to get coordinates
def get_latitudelongitude(neighborhood):
    # initialize your variable to None
    latitude_longitude_coordinates = None
    # loop until you get the coordinates
    while(latitude_longitude_coordinates is None):
        g = geocoder.arcgis('{}, Kuala Lumpur, Malaysia'.format(neighborhood))
        latitude_longitude_coordinates = g.latlng
    return latitude_longitude_coordinates

In [48]:
# call the function to get the coordinates, store in a new list using list comprehension
coordinates = [ get_latitudelongitude(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]

In [49]:
coordinates

[[3.0576900000000364, 101.74388000000005],
 [3.1484921138724, 101.69672653590337],
 [3.1903500000000236, 101.62545000000006],
 [3.1039100000000417, 101.71226000000007],
 [3.072750000000042, 101.71461000000005],
 [3.08280000000002, 101.72281000000004],
 [3.1292000000000257, 101.67844000000008],
 [3.1347800000000348, 101.67262000000005],
 [3.111020000000053, 101.66283000000004],
 [3.09898000000004, 101.73499000000004],
 [3.1357600000000616, 101.70837000000006],
 [3.12916000000007, 101.68406000000004],
 [3.147770000000037, 101.70855000000006],
 [3.057800000000043, 101.68965000000009],
 [3.1434800000000678, 101.64433000000008],
 [3.152017197420035, 101.70102760046613],
 [3.129290000000026, 101.69892000000004],
 [3.17381000000006, 101.68276000000009],
 [3.061870000000056, 101.74675000000008],
 [3.163590000000056, 101.69811000000004],
 [3.147980000000075, 101.66798000000006],
 [3.1387586696676304, 101.6840455304707],
 [3.1387586696676304, 101.6840455304707],
 [3.156685175474611, 101.69807679

In [52]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coordinates = pd.DataFrame(coordinates, columns=['Latitude', 'Longitude'])

In [53]:
# merge the coordinates into the original dataframe
kl_df['Latitude'] = df_coordinates['Latitude']
kl_df['Longitude'] = df_coordinates['Longitude']

In [54]:
# check the neighborhoods and the coordinates
print(kl_df.shape)
kl_df

(71, 3)


,Neighborhood,Latitude,Longitude
0,Alam Damai,3.057690,101.743880
1,"Ampang, Kuala Lumpur",3.148492,101.696727
2,Bandar Menjalara,3.190350,101.625450
3,Bandar Sri Permaisuri,3.103910,101.712260
4,Bandar Tasik Selatan,3.072750,101.714610
5,Bandar Tun Razak,3.082800,101.722810
6,Bangsar,3.129200,101.678440
7,Bangsar Park,3.134780,101.672620
8,Bangsar South,3.111020,101.662830
9,Batu 11 Cheras,3.098980,101.734990


In [56]:
# save the DataFrame as CSV file
kl_df.to_csv("kl_df.csv", index=False)

##### Creating a KUALALUMPUR map with nieghbourhoods on top of it.

In [58]:
# get the coordinates of KualaLumpur
address = 'Kuala Lumpur, Malaysia'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
lat = location.latitude
lng = location.longitude
print('The geograpical coordinate of Kuala Lumpur, Malaysiae {}, {}.'.format(lat, lng))

The geograpical coordinate of Kuala Lumpur, Malaysiae 3.1516636, 101.6943028.


In [65]:
# create map of KualaLumpur using latitude and longitude values
map_KL = folium.Map(location=[lat, lng], zoom_start=12)

# add markers to map
for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_KL)  
    
map_KL

In [66]:
# Saving the map as a HTML file
map_KL.save('map_KL.html')

##### Use the Foursquare API to explore various KualaLumpur neighbourhoods.

In [67]:
CLIENT_ID = 'GG5YY00NASOWDY5MNOWJYEVYNRTWKKUVEH3HB0YSNFQW5OIH' 
CLIENT_SECRET = 'RVLOVFXKHVLHOVBHYQ2BGMIAYFLCZYA3AUBTKN5X5FO5IFXG' 
VERSION = '20200129' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GG5YY00NASOWDY5MNOWJYEVYNRTWKKUVEH3HB0YSNFQW5OIH
CLIENT_SECRET:RVLOVFXKHVLHOVBHYQ2BGMIAYFLCZYA3AUBTKN5X5FO5IFXG


##### Getting top 100 venues within 2000 metres radius.

In [69]:
radius = 2000
LIMIT = 100

venues = []

for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        lng,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            lng, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [70]:
# convert the newfound venues list into a new pdDataFrame
venuesdf = pd.DataFrame(venues)

# define the column names
venuesdf.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venuesdf.shape)
venuesdf.head()

(7084, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Alam Damai,3.05769,101.74388,Machi Noodle 妈子面,3.057695,101.746635,Noodle House
1,Alam Damai,3.05769,101.74388,Pengedar Shaklee Kuala Lumpur,3.061235,101.740696,Supplement Shop
2,Alam Damai,3.05769,101.74388,628火焰鑫茶室,3.058442,101.747947,Chinese Restaurant
3,Alam Damai,3.05769,101.74388,Ivy Sekinchan Seafood Noodle House 适耕莊特制魚丸海鲜面,3.065749,101.748718,Noodle House
4,Alam Damai,3.05769,101.74388,Minang Tomyam,3.057185,101.749812,Seafood Restaurant


In [71]:
venuesdf.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Alam Damai,100,100,100,100,100,100
"Ampang, Kuala Lumpur",100,100,100,100,100,100
Bandar Menjalara,100,100,100,100,100,100
Bandar Sri Permaisuri,100,100,100,100,100,100
Bandar Tasik Selatan,99,99,99,99,99,99
Bandar Tun Razak,100,100,100,100,100,100
Bangsar,100,100,100,100,100,100
Bangsar Park,100,100,100,100,100,100
Bangsar South,100,100,100,100,100,100


##### Finind number of unique values for each result.

In [74]:
print('There are {} uniques categories.'.format(len(venuesdf['VenueCategory'].unique())))

There are 308 uniques categories.


In [76]:
##### Printing out the list of categories.

In [77]:
venuesdf['VenueCategory'].unique()[:50]

array(['Noodle House', 'Supplement Shop', 'Chinese Restaurant',
       'Seafood Restaurant', 'Restaurant', 'Breakfast Spot',
       'Vegetarian / Vegan Restaurant', 'Food Court',
       'Other Great Outdoors', 'Coffee Shop', 'Park', 'Bubble Tea Shop',
       'Dim Sum Restaurant', 'Asian Restaurant', 'Indian Restaurant',
       'Pet Store', 'Food Truck', 'Spa', 'Japanese Restaurant',
       'Chinese Breakfast Place', 'Snack Place', 'Convenience Store',
       'Dessert Shop', 'Farmers Market', 'Bakery', 'Café', 'Outlet Store',
       'Malay Restaurant', 'Steakhouse', 'Cantonese Restaurant',
       'Gym / Fitness Center', 'Fast Food Restaurant',
       'Middle Eastern Restaurant', 'Hakka Restaurant',
       'Athletics & Sports', 'Badminton Court', 'Burger Joint',
       'Mamak Restaurant', 'Winery', 'College Bookstore', 'Grocery Store',
       'Halal Restaurant', 'Gym', 'Hostel', 'South Indian Restaurant',
       'Exhibit', 'Chettinad Restaurant', 'Juice Bar', 'Hotel',
       'Speakeasy']

In [78]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venuesdf['VenueCategory'].unique()

False

##### Analyzing each neighbourhood.

In [79]:
# one hot encoding
kl_onehot = pd.get_dummies(venuesdf[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venuesdf['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(7084, 309)


,Neighborhoods,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Business Service,Cafeteria,Café,Camera Store,Campground,Candy Store,Cantonese Restaurant,Casino,Chettinad Restaurant,Chinese Breakfast Place,Chinese Restaurant,Circus,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dive Shop,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Exhibit,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Hill,Historic Site,History Museum,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kushikatsu Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Lounge,Malay Restaurant,Mamak Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Other Great Outdoors,Other Nightlife,Outdoor Event Space,Outlet Store,Pakistani Restaurant,Palace,Park,Parking,Performing Arts Venue,Pet Café,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Public Art,Racetrack,Ramen Restaurant,Recording Studio,Recreation Center,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Temple,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Th

##### Grouping rows by neighborhoods by taking the mean of frequency of occurrence of each category.

In [81]:
klgrouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(klgrouped.shape)
klgrouped

(71, 309)


,Neighborhoods,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Business Service,Cafeteria,Café,Camera Store,Campground,Candy Store,Cantonese Restaurant,Casino,Chettinad Restaurant,Chinese Breakfast Place,Chinese Restaurant,Circus,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dive Shop,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Exhibit,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Hill,Historic Site,History Museum,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kushikatsu Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Lounge,Malay Restaurant,Mamak Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Other Great Outdoors,Other Nightlife,Outdoor Event Space,Outlet Store,Pakistani Restaurant,Palace,Park,Parking,Performing Arts Venue,Pet Café,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Public Art,Racetrack,Ramen Restaurant,Recording Studio,Recreation Center,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Temple,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Th

In [82]:
len(klgrouped[klgrouped["Shopping Mall"] > 0])

38

##### Creating a new Dataframe for shopping mall entry.

In [85]:
klmall = klgrouped[["Neighborhoods","Shopping Mall"]]
klmall.head()

,Neighborhoods,Shopping Mall
0,Alam Damai,0.000000
1,"Ampang, Kuala Lumpur",0.010000
2,Bandar Menjalara,0.010000
3,Bandar Sri Permaisuri,0.000000
4,Bandar Tasik Selatan,0.010101


##### Neighbourhood Clustering.

##### Using k-means clustering for neighborhoods in KualaLumpur into 3 clusters.

In [89]:
kclusters = 3

klclustering = klmall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(klclustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 0, 0, 2, 0, 2, 1, 1, 0, 2], dtype=int32)

##### Creating a new dataframe that includes cluster as well as the top 10 venues for each neighborhood.

In [107]:
klmerged = klmall.copy()

# add clustering labels
klmerged["Cluster Labels"] = kmeans.labels_

klmerged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
klmerged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Alam Damai,0.000000,2
1,"Ampang, Kuala Lumpur",0.010000,0
2,Bandar Menjalara,0.010000,0
3,Bandar Sri Permaisuri,0.000000,2
4,Bandar Tasik Selatan,0.010101,0


In [108]:
klmerged = klmerged.join(kl_df.set_index("Neighborhood"), on="Neighborhood")

print(klmerged.shape)
klmerged.head()

(71, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Alam Damai,0.000000,2,3.057690,101.743880
1,"Ampang, Kuala Lumpur",0.010000,0,3.148492,101.696727
2,Bandar Menjalara,0.010000,0,3.190350,101.625450
3,Bandar Sri Permaisuri,0.000000,2,3.103910,101.712260
4,Bandar Tasik Selatan,0.010101,0,3.072750,101.714610


##### Sorting the results by Cluster Labels.

In [109]:
print(klmerged.shape)
klmerged.sort_values(["Cluster Labels"], inplace=True)
klmerged

(71, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
70,Wangsa Maju,0.010000,0,3.203910,101.737190
22,"Damansara, Kuala Lumpur",0.010000,0,3.138759,101.684046
23,Dang Wangi,0.020000,0,3.156685,101.698077
51,Sri Hartamas,0.020000,0,3.162200,101.650360
25,"Federal Hill, Kuala Lumpur",0.010000,0,3.136370,101.685640
27,"Jalan Cochrane, Kuala Lumpur",0.020000,0,3.132903,101.724678
28,Jalan Duta,0.010000,0,3.180086,101.677854
30,KL Eco City,0.020000,0,3.117140,101.673880
21,Damansara Town Centre,0.010000,0,3.138759,101.684046
31,"Kampung Baru, Kuala Lumpur",0.020000,0,3.165460,101.710280


##### Visualization of Clusters

In [111]:
# create map
mapclusters = folium.Map(location=[lat, lng], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lng, poi, cluster in zip(klmerged['Latitude'], klmerged['Longitude'], klmerged['Neighborhood'], klmerged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(mapclusters)
       
mapclusters

##### Saving the map as HTML file.

In [114]:
mapclusters.save('mapclusters.html')

##### Examining individual clusters.

##### Cluster 0.

In [118]:
klmerged.loc[klmerged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
70,Wangsa Maju,0.010000,0,3.203910,101.737190
22,"Damansara, Kuala Lumpur",0.010000,0,3.138759,101.684046
23,Dang Wangi,0.020000,0,3.156685,101.698077
51,Sri Hartamas,0.020000,0,3.162200,101.650360
25,"Federal Hill, Kuala Lumpur",0.010000,0,3.136370,101.685640
27,"Jalan Cochrane, Kuala Lumpur",0.020000,0,3.132903,101.724678
28,Jalan Duta,0.010000,0,3.180086,101.677854
30,KL Eco City,0.020000,0,3.117140,101.673880
21,Damansara Town Centre,0.010000,0,3.138759,101.684046
31,"Kampung Baru, Kuala Lumpur",0.020000,0,3.165460,101.710280


##### Cluster 1

In [119]:
klmerged.loc[klmerged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
67,Taman U-Thant,0.04,1,3.157700,101.724520
6,Bangsar,0.05,1,3.129200,101.678440
36,Lembah Pantai,0.04,1,3.121202,101.663899
7,Bangsar Park,0.06,1,3.134780,101.672620
42,"Pudu, Kuala Lumpur",0.04,1,3.133540,101.713070
66,Taman Tun Dr Ismail,0.03,1,3.152830,101.622710
11,Brickfields,0.03,1,3.129160,101.684060
15,Bukit Nanas,0.03,1,3.152017,101.701028


##### Cluster 2.

In [120]:
klmerged.loc[klmerged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
52,Sri Petaling,0.0,2,3.072600,101.682520
68,Taman Wahyu,0.0,2,3.222400,101.671730
54,Taman Bukit Maluri,0.0,2,3.200660,101.633370
63,Taman P. Ramlee,0.0,2,3.193600,101.705980
58,Taman Ibukota,0.0,2,3.212160,101.715400
56,Taman Connaught,0.0,2,3.082690,101.736890
59,Taman Len Seng,0.0,2,3.069080,101.742870
60,Taman Melati,0.0,2,3.223570,101.723990
61,Taman Midah,0.0,2,3.093590,101.728370
65,Taman Taynton View,0.0,2,3.087070,101.736810


##### Conclusion/Findings -

Most of the shopping malls are concentrated in the central area of Kuala Lumpur city.
Cluster 2 has no shopping malls. 
Cluster 1 has highest shopping malls.
Cluster 0 has moderate number of shopping malls.


Cluster 2 is the best neighbourhood cluster for opening a new shopping mall as there is very little to no competition from existing malls.
Cluster 0 neighbourhoods can also be considered as next alternative to open new shopping malls.
It is advised for Property Developers not to open new shopping malls in Cluster 1 neighbourhoods because of very high competition.
